### 🛠️ Imports

In [1]:
from __future__ import annotations
from aiworkshop_utils.standardlib_imports import os, json, base64, logging, Optional, List, Literal, pprint, glob, asyncio, datetime, date, time, timezone, ZoneInfo, uuid, dataclass
from aiworkshop_utils.thirdparty_imports import AutoTokenizer, load_dotenv, requests, BaseModel, Field, pd, cosine_similarity, plt, np, DataType, MilvusClient, DDGS, rprint
from aiworkshop_utils.custom_utils import show_pretty_json, encode_image
from aiworkshop_utils.jupyter_imports import Markdown, HTML, JSON, display, widgets
from aiworkshop_utils.openai_imports import OpenAI, Agent, Runner, InputGuardrail, GuardrailFunctionOutput, InputGuardrailTripwireTriggered, OpenAIChatCompletionsModel, AsyncOpenAI, set_tracing_disabled, ModelSettings, function_tool, trace, ResponseContentPartDoneEvent, ResponseTextDeltaEvent, RawResponsesStreamEvent, TResponseInputItem, ItemHelpers, MessageOutputItem, RunContextWrapper, input_guardrail, output_guardrail
from aiworkshop_utils import config

### Pydantic Model

In [2]:
class HealthData(BaseModel):
    age: int                                # Age in years
    gender: str                             # e.g. male, female, diverse
    weight: float                           # Weight in kg       
    height: float                           # Height in cm
    allergies: Optional[str]                # e.g. nuts, gluten
    eating_habits: str                      # e.g. vegetarian, vegan, omnivore, paleo
    goal: str                               # e.g. weight loss, muscle gain, maintenance
    activity_level: str                     # e.g. sedentary, lightly active, moderately active, very active
    timeCooking: int                        # Time spent cooking per day in minutes
    healthCondition: Optional[str]          # e.g. diabetes, hypertension